# Primary School Rankings

There is no official primary school ranking done by the Ministry of Education. However, we adopt the "wisdom of the crowd" and use application details in 2021 to determine a prmary schools popularity.

Popularity is calculated as total applications recieved over all phases divided by the number of vacancies.

In [18]:
# import libs

import requests
import pandas as pd
import re
from MyCreds.mycreds import Capstone_AWS_PG
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
import time
import geopandas as gpd
from geoalchemy2 import Geometry, Geography



In [19]:

geolocator = Nominatim(user_agent="pschools_singapore")

def building_to_data(search_string):
  '''
  function to query onemap api for building name - adapted from pcode_to_data
  '''
  page = 1
  results = []
    
  while True:
      try:
          response = requests.get('http://developers.onemap.sg/commonapi/search?searchVal={0}&returnGeom=Y&getAddrDetails=Y&pageNum={1}'
                                  .format(search_string, page)) \
                              .json()
      except requests.exceptions.ConnectionError as e:
          print('Fetching {} failed. Retrying in 2 sec'.format(search_string))
          
          time.sleep(2)
          continue
          
      results = results + response['results']
      if response['totalNumPages'] > page:
          page = page + 1
      else:
          break
          
  return results

# grab application table from website
url = 'https://sgschooling.com/year/2021/all'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

# load into dataframe
primary_ranks = pd.read_html(r.text)[0]
primary_ranks

,School,Phase 1,2A(1),2A(2),2B,2C,2C(S),3
0,Admiralty,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,↳ Vacancy (210),170.0,87.0,87.0,50,50,0,0
2,↳ Applied,83.0,0.0,28.0,60,93,0,-
3,↳ Taken,83.0,0.0,28.0,50SC1-2,50SC<1,0,-
4,Ahmad Ibrahim,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
719,↳ Taken,65.0,0.0,71.0,0,144SC<1#,0,-
720,Northshore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
721,↳ Vacancy (240),200.0,194.0,194.0,117,233,140,61
722,↳ Applied,6.0,0.0,0.0,1,93,79,-


## Clean the data and calculate popularity

In [20]:
# clean and process data

# set up some empty lists
schools = []
vacancy = []
applied = []

# iterate through each row of the dataframe
for i in primary_ranks.iterrows():
    # if it doesn't start with a ↳ it is the name of the school
    if not i[1][0].startswith("↳"):
        schools.append(i[1][0])
    # vacancies for each school is stored in within the (xxx)
    elif i[1][0].startswith("↳ Vacancy"):
        vacancy.append(re.findall(r'\d+',i[1][0])[0])
    # applications for each phase are in Applied
    elif i[1][0].startswith("↳ Applied"):
        applied_per_phase = []
        # if we are in the applied row
        for idx,num in enumerate(i[1]):
            try:
                # add the numbers to a list except the first
                applied_per_phase.append(int(num))
            except ValueError:
                # error when num is a '-' and not zero/or convertable to int like at idx 0
                pass
        applied.append(sum(applied_per_phase))

# put data into a data frame
primary_ranks_cleaned = pd.DataFrame(
    {
        'school_name' : schools,
        'vacancies' : vacancy,
        'total_application' : applied
    }
)

# add popularity
primary_ranks_cleaned['popularity'] = primary_ranks_cleaned['total_application'].astype(int)/primary_ranks_cleaned['vacancies'].astype(int)

# clean school name to match strings
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['school_name'].str.upper()
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('PRIMARY','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('SCHOOL','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('’','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('\'','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('.','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace(' ','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('(','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace(')','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.strip()


primary_ranks_cleaned

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,school_name,vacancies,total_application,popularity,match_on
0,Admiralty,210,264,1.257143,ADMIRALTY
1,Ahmad Ibrahim,210,84,0.400000,AHMADIBRAHIM
2,Ai Tong,300,364,1.213333,AITONG
3,Alexandra,200,223,1.115000,ALEXANDRA
4,Anchor Green,240,271,1.129167,ANCHORGREEN
...,...,...,...,...,...
176,Zhangde,210,223,1.061905,ZHANGDE
177,Zhenghua,240,253,1.054167,ZHENGHUA
178,Zhonghua,180,83,0.461111,ZHONGHUA
179,Valour,280,295,1.053571,VALOUR


## Sanity Checking

In [21]:
primary_ranks_cleaned.sort_values(by=['popularity'],ascending=False).head(20)

,school_name,vacancies,total_application,popularity,match_on
113,Princess Elizabeth,200,392,1.960000,PRINCESSELIZABETH
100,Northland,240,413,1.720833,NORTHLAND
92,Nan Hua,240,391,1.629167,NANHUA
91,Nan Chiau,300,469,1.563333,NANCHIAU
66,Horizon,240,362,1.508333,HORIZON
31,CHIJ St. Nicholas Girls’,210,315,1.500000,CHIJSTNICHOLASGIRLS
141,St. Hilda’s,270,402,1.488889,STHILDAS
112,Poi Ching,240,357,1.487500,POICHING
58,Gongshang,210,310,1.476190,GONGSHANG
152,Temasek,200,290,1.450000,TEMASEK


There appears to be some nuances by phase but generally looks ok to use as a ranking - most searches of primary school ranking only consider certain [phases](https://sgschooling.com/phases) of the application process. 

## Add a percentile rank column to group schools

In [22]:
primary_ranks_cleaned['percentile_rank'] = primary_ranks_cleaned['popularity'].rank(pct=True)
primary_ranks_cleaned

,school_name,vacancies,total_application,popularity,match_on,percentile_rank
0,Admiralty,210,264,1.257143,ADMIRALTY,0.878453
1,Ahmad Ibrahim,210,84,0.400000,AHMADIBRAHIM,0.022099
2,Ai Tong,300,364,1.213333,AITONG,0.834254
3,Alexandra,200,223,1.115000,ALEXANDRA,0.596685
4,Anchor Green,240,271,1.129167,ANCHORGREEN,0.649171
...,...,...,...,...,...,...
176,Zhangde,210,223,1.061905,ZHANGDE,0.480663
177,Zhenghua,240,253,1.054167,ZHENGHUA,0.439227
178,Zhonghua,180,83,0.461111,ZHONGHUA,0.049724
179,Valour,280,295,1.053571,VALOUR,0.430939


In [23]:
schools = 'data/school-directory-and-information/general-information-of-schools.csv'

all_schools = pd.read_csv(schools)

primary = all_schools[(all_schools['mainlevel_code']=='PRIMARY')]
primary


,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na
5,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A Prince Charles Crescent,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
6,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 Anchorvale Drive,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
338,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,na
341,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
342,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil


In [24]:
# clean school name to match strings
primary['match_on'] = primary['school_name'].str.replace('PRIMARY','')
primary['match_on'] = primary['match_on'].str.replace('SCHOOL','')
primary['match_on'] = primary['match_on'].str.replace("'",'')
primary['match_on'] = primary['match_on'].str.replace(".",'')
primary['match_on'] = primary['match_on'].str.replace(' ','')
primary['match_on'] = primary['match_on'].str.replace('(','')
primary['match_on'] = primary['match_on'].str.replace(')','')
primary['match_on'] = primary['match_on'].str.strip()
primary

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ADMIRALTY
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,AHMADIBRAHIM
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na,AITONG
5,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A Prince Charles Crescent,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ALEXANDRA
6,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 Anchorvale Drive,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ANCHORGREEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,YUHUA
338,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,na,YUMIN
341,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHANGDE
342,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHENGHUA


In [25]:
# merge the 2 dataframes
primary_ranked = primary.merge(primary_ranks_cleaned, left_on='match_on',right_on='match_on',how='left',indicator=True)
primary_ranked

,school_name_x,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on,school_name_y,vacancies,total_application,popularity,percentile_rank,_merge
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,Chinese,Malay,Tamil,ADMIRALTY,Admiralty,210,264.0,1.257143,0.878453,both
1,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,Chinese,Malay,Tamil,AHMADIBRAHIM,Ahmad Ibrahim,210,84.0,0.400000,0.022099,both
2,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,Chinese,na,na,AITONG,Ai Tong,300,364.0,1.213333,0.834254,both
3,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A Prince Charles Crescent,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,Chinese,Malay,Tamil,ALEXANDRA,Alexandra,200,223.0,1.115000,0.596685,both
4,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 Anchorvale Drive,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,Chinese,Malay,Tamil,ANCHORGREEN,Anchor Green,240,271.0,1.129167,0.649171,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,Chinese,Malay,Tamil,YUHUA,Yuhua,240,165.0,0.687500,0.149171,both
179,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,Chinese,Malay,na,YUMIN,Yumin,180,191.0,1.061111,0.469613,both
180,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,Chinese,Malay,Tamil,ZHANGDE,Zhangde,210,223.0,1.061905,0.480663,both
181,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,Chinese,Malay,Tamil,ZHENGHUA,Zhenghua,240,253.0,1.054167,0.439227,both


In [26]:
primary_ranked[primary_ranked['_merge']!='both']

,school_name_x,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on,school_name_y,vacancies,total_application,popularity,percentile_rank,_merge
44,EUNOS PRIMARY SCHOOL,http://www.eunospri.moe.edu.sg/,95 JALAN EUNOS,419529,67463336,na,67488554,na,eunos_ps@moe.edu.sg,Nearest MRT Station - Eunos,...,Chinese,Malay,Tamil,EUNOS,NaN,NaN,NaN,NaN,NaN,left_only
61,GUANGYANG PRIMARY SCHOOL,http://www.guangyangpri.moe.edu.sg,6 Bishan Street 12,579806,62580455,na,62580010,na,GYPS@MOE.EDU.SG,Bishan MRT,...,Chinese,Malay,Tamil,GUANGYANG,NaN,NaN,NaN,NaN,NaN,left_only
75,JUYING PRIMARY SCHOOL,http://www.juyingpri.moe.edu.sg/,31 JURONG WEST STREET 91,649037,67932039,na,67932040,na,JUYING_PS@MOE.EDU.SG,Boon Lay MRT Station (Jurong Point),...,Chinese,Malay,Tamil,JUYING,NaN,NaN,NaN,NaN,NaN,left_only
113,PIONEER PRIMARY SCHOOL,http://www.pioneerpri.moe.edu.sg,23 JURONG WEST STREET 81,649076,67932397,na,67933974,na,pioneer_ps@moe.edu.sg,Boon Lay Station,...,Chinese,Malay,Tamil,PIONEER,NaN,NaN,NaN,NaN,NaN,left_only
147,STAMFORD PRIMARY SCHOOL,http://www.stamfordpri.moe.edu.sg,1 VICTORIA LANE,198423,62945074,na,62919262,na,STAMFORD_PS@MOE.EDU.SG,"Bugis MRT Station, Jalan Besar MRT Station",...,Chinese,Malay,Tamil,STAMFORD,NaN,NaN,NaN,NaN,NaN,left_only


In [27]:
primary_ranked[['vacancies','total_application','popularity','percentile_rank']]= primary_ranked[['vacancies','total_application','popularity','percentile_rank']].fillna(value=0)
primary_ranked.drop(columns=['_merge'],inplace=True)
primary_ranked

,school_name_x,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on,school_name_y,vacancies,total_application,popularity,percentile_rank
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,No,Chinese,Malay,Tamil,ADMIRALTY,Admiralty,210,264.0,1.257143,0.878453
1,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,No,Chinese,Malay,Tamil,AHMADIBRAHIM,Ahmad Ibrahim,210,84.0,0.400000,0.022099
2,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,No,Chinese,na,na,AITONG,Ai Tong,300,364.0,1.213333,0.834254
3,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A Prince Charles Crescent,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,No,Chinese,Malay,Tamil,ALEXANDRA,Alexandra,200,223.0,1.115000,0.596685
4,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 Anchorvale Drive,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,No,Chinese,Malay,Tamil,ANCHORGREEN,Anchor Green,240,271.0,1.129167,0.649171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,No,Chinese,Malay,Tamil,YUHUA,Yuhua,240,165.0,0.687500,0.149171
179,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,No,Chinese,Malay,na,YUMIN,Yumin,180,191.0,1.061111,0.469613
180,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,No,Chinese,Malay,Tamil,ZHANGDE,Zhangde,210,223.0,1.061905,0.480663
181,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,No,Chinese,Malay,Tamil,ZHENGHUA,Zhenghua,240,253.0,1.054167,0.439227


In [28]:
# use geocode to get location
sch_to_geocode = primary_ranked['school_name_x'] + ' Singapore '
add_to_geocode = primary_ranked['address'] + ' Singapore ' + primary_ranked['postal_code'].astype(str)
add_loc_cols = {'latitude':[],'longitude':[]}

for sch,add in zip(sch_to_geocode,add_to_geocode):
    # print(sch)
    # print(add)
    try:
        geo = geolocator.geocode(sch)[1]
    except TypeError:
        try:
            geo = geolocator.geocode(add)[1]
        except TypeError:
            query = building_to_data(sch)[0]
            print(query['SEARCHVAL'])
            geo = [query['LATITUDE'].replace("'",''),query['LONGITUDE'].replace("'",'')]
    # print(geo)
    add_loc_cols['latitude'].append(geo[0])
    add_loc_cols['longitude'].append(geo[1])
add_loc_cols

GEYLANG METHODIST SCHOOL (PRIMARY)
LIANHUA PRIMARY SCHOOL


{'latitude': [1.4429414,
  1.4338487,
  1.3607125500000001,
  1.2912837000000001,
  1.3903797999999998,
  1.37631785,
  1.3691002,
  1.30928255,
  1.31875235,
  1.3635823,
  1.3841149499999998,
  1.3240565000000002,
  1.3223157,
  1.27640575,
  1.3428882,
  1.37341685,
  1.33751905,
  1.34570095,
  1.45085035,
  1.3266014,
  1.2758142000000001,
  1.3728079,
  1.3355312499999998,
  1.33982335,
  1.30643785,
  1.274664,
  1.3577786,
  1.37311065,
  1.36644755,
  1.3331269,
  1.4387276,
  1.35054425,
  1.37793775,
  1.31556955,
  1.3945064,
  1.38021795,
  1.35142845,
  1.3354514,
  1.3591826,
  1.3950086000000002,
  1.3528815,
  1.40035045,
  1.37491195,
  1.4544299500000002,
  1.32481685,
  1.44383025,
  1.30081995,
  1.3125247,
  1.3300987,
  1.3976461,
  1.3899632999999998,
  1.3404649,
  1.3369298,
  1.43072155,
  1.3364853499999998,
  1.2858741500000002,
  '1.31759126757284',
  1.3573263,
  1.39620005,
  1.3856168,
  1.4399652,
  1.34577845,
  1.31173415,
  1.3170568999999999,
  1.3

In [29]:
primary_ranked = primary_ranked.assign(**add_loc_cols)
primary_ranked

,school_name_x,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,mothertongue2_code,mothertongue3_code,match_on,school_name_y,vacancies,total_application,popularity,percentile_rank,latitude,longitude
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,Malay,Tamil,ADMIRALTY,Admiralty,210,264.0,1.257143,0.878453,1.442941,103.800345
1,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,Malay,Tamil,AHMADIBRAHIM,Ahmad Ibrahim,210,84.0,0.400000,0.022099,1.433849,103.83271
2,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,na,na,AITONG,Ai Tong,300,364.0,1.213333,0.834254,1.360713,103.833018
3,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A Prince Charles Crescent,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,Malay,Tamil,ALEXANDRA,Alexandra,200,223.0,1.115000,0.596685,1.291284,103.824066
4,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 Anchorvale Drive,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,Malay,Tamil,ANCHORGREEN,Anchor Green,240,271.0,1.129167,0.649171,1.39038,103.887354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,Malay,Tamil,YUHUA,Yuhua,240,165.0,0.687500,0.149171,1.342891,103.74096
179,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,Malay,na,YUMIN,Yumin,180,191.0,1.061111,0.469613,1.351264,103.950874
180,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,Malay,Tamil,ZHANGDE,Zhangde,210,223.0,1.061905,0.480663,1.28425,103.826082
181,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,Malay,Tamil,ZHENGHUA,Zhenghua,240,253.0,1.054167,0.439227,1.379426,103.769666


In [30]:
primary_ranked = gpd.GeoDataFrame(primary_ranked, geometry=gpd.points_from_xy(primary_ranked.longitude,primary_ranked.latitude,))
primary_ranked

,school_name_x,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,mothertongue3_code,match_on,school_name_y,vacancies,total_application,popularity,percentile_rank,latitude,longitude,geometry
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,Tamil,ADMIRALTY,Admiralty,210,264.0,1.257143,0.878453,1.442941,103.800345,POINT (103.80035 1.44294)
1,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,Tamil,AHMADIBRAHIM,Ahmad Ibrahim,210,84.0,0.400000,0.022099,1.433849,103.83271,POINT (103.83271 1.43385)
2,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,na,AITONG,Ai Tong,300,364.0,1.213333,0.834254,1.360713,103.833018,POINT (103.83302 1.36071)
3,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A Prince Charles Crescent,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,Tamil,ALEXANDRA,Alexandra,200,223.0,1.115000,0.596685,1.291284,103.824066,POINT (103.82407 1.29128)
4,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 Anchorvale Drive,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,Tamil,ANCHORGREEN,Anchor Green,240,271.0,1.129167,0.649171,1.39038,103.887354,POINT (103.88735 1.39038)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,Tamil,YUHUA,Yuhua,240,165.0,0.687500,0.149171,1.342891,103.74096,POINT (103.74096 1.34289)
179,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,na,YUMIN,Yumin,180,191.0,1.061111,0.469613,1.351264,103.950874,POINT (103.95087 1.35126)
180,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,Tamil,ZHANGDE,Zhangde,210,223.0,1.061905,0.480663,1.28425,103.826082,POINT (103.82608 1.28425)
181,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,Tamil,ZHENGHUA,Zhenghua,240,253.0,1.054167,0.439227,1.379426,103.769666,POINT (103.76967 1.37943)


In [31]:
# declare CRS
primary_ranked.crs = "EPSG:4326"

In [32]:
primary_ranked.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [33]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)
sql_alc_cnxn = engine.connect()


In [34]:
primary_ranked.to_postgis('primary_schools_ranked_with_loc',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT', srid= 4326)})